In [1]:
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
%matplotlib inline
pd.set_option('display.max_colwidth', 300)

In [3]:
meta = pd.read_excel("articoli.xlsx", encoding="utf8", error_bad_lines=False)
meta = meta.replace(np.nan, '', regex=True)

In [4]:
meta.head()

,ID,Titolo,Url,Data,Abstract,Fonte,Stato,Tecnologia 1,Tecnologia 2,Tecnologia 3,...,Application 2,Application 3,Trend 1,Trend 2,Trend 3,Unnamed: 22,Trend,Tecnologia,Vertical,Settore
0,655dde2aeba917a7ca29388dbc3d77cb,"A Colorado Neighborhood Runs on a Self-Reliant, Clean Energy Grid",https://futurism.com/colorado-neighborhood-built-clean-energy-grid,,"Local Grid In Basalt , Colorado , a new housing development is testing out a new way to distribute electricity . Instead of relying on a plant somewhere to dole out electricity to their homes , residents of the Basalt Vista development are relying on an experimental virtual power plant.",Futurism,DONE,Renewable Energies,Neural Networks,,...,Smart Contracts,,Low Emissions,Smart Grids,Electrification,,,,,
1,ec86361327808e44cb6ede901b8ae718,A New Hole Is Forming in the Ozone Layer Over the Arctic,https://futurism.com/hole-ozone-layer-arctic,,"Over the last month , a new hole in the ozone layer has started to form over the Arctic . But this time around , extreme weather and atmospheric conditions have led to a far greater depletion than normal , according to a European Space Agency press release .",Futurism,DONE,Renewable Energies,,,...,,,Low Emissions,,,,,,,
2,d0d9e4d9fd83756aea7505900b50b8da,"Here's NASA's Plan for a Lunar ""Base Camp""",https://futurism.com/nasa-artemis-base-camp-moon,,"NASA has outlined its plans for a lunar station called the Artemis Base Camp in a new 13-page report titled “NASA’ s Plan for Sustained Lunar Exploration and Development” submitted to the National Space Council last week . After 20 years of continuously living in low-Earth orbit , we’re now read...",Futurism,DONE,Low-Earth Satellites,,,...,,,Space Economy,,,,,,,
3,6b5272a2204d6dbfb30bddb91642a620,Is Residential Solar Power Worth It? This Free Online Tool Will Tell You.,https://futurism.com/is-residential-solar-power-worth-it,,"Everybody knows solar power is great in theory . Over the last 10 years the cost of installing solar panels has dropped roughly 70 percent thanks to lower equipment costs , lower installation costs , and various government tax incentives , including the Federal Solar Tax Credit . After that , a ...",Futurism,DONE,Renewable Energies,,,...,,,Decarbonization,,,,,,,
4,7e006110e942ff2f17f7ef5ba8191fa4,NASA Funds Giant Radio Telescope Concept on Far Side of Moon,https://futurism.com/nasa-radio-telescope-far-side-moon,,"NASA is funding research for a giant radio telescope in a crater on the far side of the Moon . According to a description by research lead Saptarshi Bandyopadhyay , robotics technologist at NASA ’ s Jet Propulsion Laboratory in California , the idea is to capture the “ tremendous advantages comp...",Futurism,DONE,Low-Earth Satellites,,,...,,,,,,,,,,


In [5]:
article_id = meta['ID']
len(article_id)

104

In [6]:
meta['Tecnologia 1'][0]

'Renewable Energies'

In [ ]:
#tech1 = []
#for i in meta['Tecnologia 1']:
#    tech1.append(i)
#
#tech2 = []
#for i in meta['Tecnologia 2']:
#    tech2.append(i)
#
#tech3 = []
#for i in meta['Tecnologia 3']:
#    tech3.append(i)

In [7]:
#type(tech1)

In [8]:
tech1 = []
for i in meta['Tecnologia 1']:
    tech1.append(i)
#   if len(i): tech1.append(i + ', ')
#   else: tech1.append('')

#tech2 = []
#for i in meta['Tecnologia 2']:
#    if len(i): tech2.append(i + ', ')  
#    else: tech2.append('')
#
#tech3 = []
#for i in meta['Tecnologia 3']:
#    if len(i): tech3.append(i + ', ') 
#    else: tech3.append('')
#        
#len(tech1)

In [9]:
Tech = []
for i in range(len(tech1)):
    l = []
    if len(tech1[i]): l.append(tech1[i])
#    if len(tech2[i]): l.append(tech2[i])
#    if len(tech3[i]): l.append(tech3[i])
    Tech.append(list(l))
    
Tech[0]

['Renewable Energies']

In [10]:
# Create Tech Dataframe
tech = pd.DataFrame({'ID': article_id, 'Tech': Tech})
tech.head()

,ID,Tech
0,655dde2aeba917a7ca29388dbc3d77cb,[Renewable Energies]
1,ec86361327808e44cb6ede901b8ae718,[Renewable Energies]
2,d0d9e4d9fd83756aea7505900b50b8da,[Low-Earth Satellites]
3,6b5272a2204d6dbfb30bddb91642a620,[Renewable Energies]
4,7e006110e942ff2f17f7ef5ba8191fa4,[Low-Earth Satellites]


In [ ]:
## create dataframe
#tech = pd.DataFrame({'ID': article_id, 'tech1': tech1, 'tech2': tech2,'tech3': tech3})
##tech = pd.DataFrame({'tech1': tech1, 'tech2': tech2,'tech3': tech3})
#tech.head()

In [ ]:
#cols = ['tech1', 'tech2', 'tech3']
#tech['Tech'] = tech[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
#tech = tech.drop(cols, axis=1)
#tech.head()

In [11]:
articoli = meta[['ID','Titolo','Abstract']]
articoli = pd.merge(articoli, tech, on = 'ID')

In [12]:
articoli.head()

,ID,Titolo,Abstract,Tech
0,655dde2aeba917a7ca29388dbc3d77cb,"A Colorado Neighborhood Runs on a Self-Reliant, Clean Energy Grid","Local Grid In Basalt , Colorado , a new housing development is testing out a new way to distribute electricity . Instead of relying on a plant somewhere to dole out electricity to their homes , residents of the Basalt Vista development are relying on an experimental virtual power plant.",[Renewable Energies]
1,ec86361327808e44cb6ede901b8ae718,A New Hole Is Forming in the Ozone Layer Over the Arctic,"Over the last month , a new hole in the ozone layer has started to form over the Arctic . But this time around , extreme weather and atmospheric conditions have led to a far greater depletion than normal , according to a European Space Agency press release .",[Renewable Energies]
2,d0d9e4d9fd83756aea7505900b50b8da,"Here's NASA's Plan for a Lunar ""Base Camp""","NASA has outlined its plans for a lunar station called the Artemis Base Camp in a new 13-page report titled “NASA’ s Plan for Sustained Lunar Exploration and Development” submitted to the National Space Council last week . After 20 years of continuously living in low-Earth orbit , we’re now read...",[Low-Earth Satellites]
3,6b5272a2204d6dbfb30bddb91642a620,Is Residential Solar Power Worth It? This Free Online Tool Will Tell You.,"Everybody knows solar power is great in theory . Over the last 10 years the cost of installing solar panels has dropped roughly 70 percent thanks to lower equipment costs , lower installation costs , and various government tax incentives , including the Federal Solar Tax Credit . After that , a ...",[Renewable Energies]
4,7e006110e942ff2f17f7ef5ba8191fa4,NASA Funds Giant Radio Telescope Concept on Far Side of Moon,"NASA is funding research for a giant radio telescope in a crater on the far side of the Moon . According to a description by research lead Saptarshi Bandyopadhyay , robotics technologist at NASA ’ s Jet Propulsion Laboratory in California , the idea is to capture the “ tremendous advantages comp...",[Low-Earth Satellites]


In [13]:
# remove samples with no tags
articoli['Tech'] = articoli['Tech'].apply(lambda y: '' if len(y)==0 else y)
articoli = articoli[~(articoli['Tech'].str.len() == 0)]
articoli = articoli[~(articoli['Abstract'].str.len() == 0)]

In [14]:
articoli.head()

,ID,Titolo,Abstract,Tech
0,655dde2aeba917a7ca29388dbc3d77cb,"A Colorado Neighborhood Runs on a Self-Reliant, Clean Energy Grid","Local Grid In Basalt , Colorado , a new housing development is testing out a new way to distribute electricity . Instead of relying on a plant somewhere to dole out electricity to their homes , residents of the Basalt Vista development are relying on an experimental virtual power plant.",[Renewable Energies]
1,ec86361327808e44cb6ede901b8ae718,A New Hole Is Forming in the Ozone Layer Over the Arctic,"Over the last month , a new hole in the ozone layer has started to form over the Arctic . But this time around , extreme weather and atmospheric conditions have led to a far greater depletion than normal , according to a European Space Agency press release .",[Renewable Energies]
2,d0d9e4d9fd83756aea7505900b50b8da,"Here's NASA's Plan for a Lunar ""Base Camp""","NASA has outlined its plans for a lunar station called the Artemis Base Camp in a new 13-page report titled “NASA’ s Plan for Sustained Lunar Exploration and Development” submitted to the National Space Council last week . After 20 years of continuously living in low-Earth orbit , we’re now read...",[Low-Earth Satellites]
3,6b5272a2204d6dbfb30bddb91642a620,Is Residential Solar Power Worth It? This Free Online Tool Will Tell You.,"Everybody knows solar power is great in theory . Over the last 10 years the cost of installing solar panels has dropped roughly 70 percent thanks to lower equipment costs , lower installation costs , and various government tax incentives , including the Federal Solar Tax Credit . After that , a ...",[Renewable Energies]
4,7e006110e942ff2f17f7ef5ba8191fa4,NASA Funds Giant Radio Telescope Concept on Far Side of Moon,"NASA is funding research for a giant radio telescope in a crater on the far side of the Moon . According to a description by research lead Saptarshi Bandyopadhyay , robotics technologist at NASA ’ s Jet Propulsion Laboratory in California , the idea is to capture the “ tremendous advantages comp...",[Low-Earth Satellites]


In [18]:
categories = list(articoli.columns.values)
categories

['ID', 'Titolo', 'Abstract', 'Tech']

In [ ]:
# function for text cleaning 
def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    
    return text

In [ ]:
articoli['Abstract'] = articoli['Abstract'].apply(lambda x: clean_text(x))
articoli.head()

In [ ]:
articoli['Tech'][2]

In [ ]:
def freq_words(x, terms = 30): 
  all_words = ' '.join([text for text in x]) 
  all_words = all_words.split() 
  fdist = nltk.FreqDist(all_words) 
  words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())}) 
  
  # selecting top 20 most frequent words 
  d = words_df.nlargest(columns="count", n = terms) 
  
  # visualize words and frequencies
  plt.figure(figsize=(12,15)) 
  ax = sns.barplot(data=d, x= "count", y = "word") 
  ax.set(ylabel = 'Word') 
  plt.show()
  
# print 100 most frequent words 
freq_words(articoli['Abstract'], 30)

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

articoli['Abstract'] = articoli['Abstract'].apply(lambda x: remove_stopwords(x))

In [ ]:
articoli['Abstract'][2]

In [ ]:
freq_words(articoli['Abstract'], 30)

In [ ]:
articoli['Tech'].head()

In [ ]:
articoli['Tech'][2]

In [ ]:
# convert text to features
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(articoli['Tech'])

In [ ]:
# transform target variable
y = multilabel_binarizer.transform(articoli['Tech'])

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)

In [ ]:
# split dataset into training and validation set
xtrain, xval, ytrain, yval = train_test_split(articoli['Abstract'], y, test_size=0.2, random_state=9)

In [ ]:
xtrain.head()

In [ ]:
ytrain

In [ ]:
#build prediction model
from sklearn.linear_model import LogisticRegression

# Binary Relevance
from sklearn.multiclass import OneVsRestClassifier

# Performance metric
from sklearn.metrics import f1_score

In [ ]:
# create TF-IDF features
xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer.transform(xval)

In [ ]:
def CLF_prediction(xtrain_tfidf, ytrain):
    lr = LogisticRegression()
    clf = OneVsRestClassifier(lr)
    
    # fit model on train data
    clf.fit(xtrain_tfidf, ytrain)
    
    # make predictions for validation set
    y_pred = clf.predict(xval_tfidf)
    
    return y_pred

In [ ]:
#lr = LogisticRegression()
#clf = OneVsRestClassifier(lr)

In [ ]:
# fit model on train data
#clf.fit(xtrain_tfidf, ytrain)

In [ ]:
# make predictions for validation set
#y_pred = clf.predict(xval_tfidf)

In [ ]:
y_pred = CLF_prediction(xtrain_tfidf, ytrain)

In [ ]:
# it is the one-hot encoded form of the unique genre tags. We will have to find a way to convert it into tech tags.
y_pred[3]

In [ ]:
#convert the predicted arrays into movie genre tags
multilabel_binarizer.inverse_transform(y_pred)[3]

In [ ]:
# evaluate performance
f1_score(yval, y_pred, average="micro")

In [ ]:
# predict probabilities
#y_pred_prob = clf.predict_proba(xval_tfidf)

In [ ]:
#t = 0.3 # threshold value
#y_pred_new = (y_pred_prob >= t).astype(int)

In [ ]:
# evaluate performance
#f1_score(yval, y_pred_new, average="micro")

In [ ]:
def infer_tags(q):
    q = clean_text(q)
    q = remove_stopwords(q)
    q_vec = tfidf_vectorizer.transform([q])
    q_pred = clf.predict(q_vec)
    return multilabel_binarizer.inverse_transform(q_pred)

In [ ]:
for i in range(5): 
    k = xval.sample(1).index[0]
    print("Article: ", articoli['Abstract'][k], \
        "\nPredicted tech: ", infer_tags(xval[k])),
    print("Actual tech: ",articoli['Tech'][k], "\n")